2.2）Diagonal / Off-diag-real / Off-diag-imag 三值标注
输出 embedding vector，值为：

2 for diagonal

1 for off-diagonal real part

0 for off-diagonal imag part
是的，三种 embedding 都可以支持不同长度的 vector，因为我们可以：

先通过长度还原出矩阵维度 D（D² = len(vec)）

按规则构造对应 embedding 向量



🟨 Embedding 2.2：Diagonal / Off-Re / Off-Im（标注为 2 / 1 / 0）
通道数 C=1，标量形式，适合整数 embedding（类似类别标签）

可用 nn.Embedding(num_classes=3, dim)，如果未来升级为 Transformer



✅ 已完成 `Embedding 2.2` 的生成函数 `generate_embedding_2_2`，详细说明如下：

---

## 📌 功能概述

为每个向量化密度矩阵生成 embedding，标记为：

* `2`: diagonal 对角项（前 D 项）
* `1`: off-diagonal 实部
* `0`: off-diagonal 虚部

输出 shape 为 `(N, L, 1)`，与原始输入 `rho.npy` 一一对应。

---

## 🧾 函数使用方法

```python
generate_embedding_2_2("your_density_matrix_vectors.npy")
```

这将自动保存为：

```
your_density_matrix_vectors_emb2.2.npy
```

如需自定义保存路径：

```python
generate_embedding_2_2("your_file.npy", save_path="your_embedding.npy")
```

---

## ✅ 下一步建议

是否继续写：

* **Embedding 2.3**（输出 `(i,j,type)` → 编码或三元组形式）
* 或者写一个 **合并输入与 embedding 的工具函数**（生成 `(N, L, 2)` 或 `(N, L, 3)` 的 CNN 输入）

你可以上传一个 `.npy` 文件让我帮你验证结果是否正确、可视化 embedding、或合成 CNN 输入格式。



In [ ]:
import numpy as np
import os

def generate_embedding_2_2(npy_file_path, save_path=None):
    """
    生成 Embedding 2.2：区分 diagonal (2), off-diagonal real part (1), off-diagonal imag part (0)

    参数：
    - npy_file_path: str，原始 .npy 文件路径，文件中每行是一个向量化的密度矩阵
    - save_path: str，可选，保存 embedding 的路径，若为 None 则自动命名为 <原文件名>_emb2.2.npy

    输出：
    - 保存一个 shape = (N, L, 1) 的 embedding 文件，其中：
        - 每个位置的值为：
            - 2：对角项
            - 1：off-diagonal 的实部
            - 0：off-diagonal 的虚部
    """
    rho = np.load(npy_file_path)  # shape: (N, L)
    N, L = rho.shape

    D = int(np.sqrt(L))
    assert D * D == L, f"向量长度 {L} 无法开方为整数，可能不是 D×D 的向量化密度矩阵"

    embedding = np.zeros((N, L, 1), dtype=np.int8)

    # Diagonal entries 在向量开头，共 D 项
    embedding[:, :D, 0] = 2

    # 剩下的位置是 off-diagonal 部分的实部 + 虚部，按对出现：
    # 对于 (i,j) 且 i < j，共 D(D-1)/2 对，每对包含 Re, Im 共 2 项
    # 即：embedding[:, D::2, 0] 是 Re，embedding[:, D+1::2, 0] 是 Im
    embedding[:, D::2, 0] = 1  # Re
    embedding[:, D+1::2, 0] = 0  # Im（默认值其实是 0，可省略，但写出来更清晰）

    # 保存路径
    if save_path is None:
        base, ext = os.path.splitext(npy_file_path)
        save_path = f"{base}_emb2.2.npy"

    np.save(save_path, embedding)
    return save_path

# 示例路径（请替换为真实文件）
# generate_embedding_2_2("/mnt/data/example_density_vectors.npy")
